# SR Example

First be sure the notebook is running with GPU.

Runtime -> Change Runtime Type -> GPU


***Le code fourni au départ est celui pour faire la Super Résolution***

# Charger les bibliothèques

In [ ]:
# Colab only
# download and install Pytorch

# ferdi 1

!pip install torch torchvision
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
import platform

print(f"Python version: {platform.python_version()}")

import torch
torch.cuda.is_available()


In [ ]:
# import the packages
import numpy as np
from PIL import Image
from IPython.display import clear_output
import matplotlib.pyplot as plt

# import PyTorch (Deep Learning lib)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import math
import inspect  # debugage
import pandas as pd

# La Base de donnée

On a choisit de travailler avec CIFAR 10, des images RGB 32 par 32, avec 10 classes, 50000 données d'entrainement et 10000 données de test.


On a voulu au départ travailler avec la base de donnée CELEBA, mais celle si étant trop lourde, avec plus de 2000 classes, on s'est retrouvé face aux problèmes suivants :
- devoir charger les données prend beaucoup de temps (+ de 200000 images)
- le téléchargement devait se faire sur un lien drive qui autorisait un download par 24h
- google collab enregistre le code mais vide les fichiers entre chaque session, on a donc tout perdu après les efforts

On a choisit une solutions sans drive, pytorch offre directement un accès à la base de donnée CIFAR10, qu'on peut donc download a chaque session sans difficulté et rapidement (car plus légère).


In [ ]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print(trainset)

print(testset)


On utilise une transformation pour normaliser les images $I \in [0, 1]^3$ de CIFAR-10 de façon à avoir $I_{norm} \in [-1, 1]^3$ :

$
I_{norm} = \frac{I - \mu}{\sigma}
$

d'ou $\mu = 0.5$ et $\sigma = 0.5$

La normalisation vise à centrer les valeurs autour de zéro et à les mettre à l'échelle de manière à avoir une variance relativement uniforme. Cela peut aider à stabiliser et à accélérer l'entraînement du modèle, en particulier lorsque vous utilisez des méthodes d'optimisation sensibles à l'échelle des données.

In [ ]:
# Fonction pour afficher le nombre d'images par classe
def show_class_distribution(dataset, set_name):
    class_counts = [0] * len(classes)
    for _, label in dataset:
        class_counts[label] += 1

    fig, ax = plt.subplots()
    ax.bar(classes, class_counts)
    ax.set_xlabel('Classes')
    ax.set_ylabel('Nombre d\'images')
    ax.set_title(f'Distribution des classes dans l\'ensemble {set_name}')
    plt.show()

# Afficher la distribution des classes dans l'ensemble d'entraînement
show_class_distribution(trainset, "d'entraînement")

# Afficher la distribution des classes dans l'ensemble de test
show_class_distribution(testset, "de test")

In [ ]:
# afficher un batch : (cette case peut être run autant de fois qu'on veut)

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)



# show images
imshow(torchvision.utils.make_grid(images))
print(f"batch_size = {images.shape[0]} et dimension d'une image = {images.shape[1:]}")
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))


# Le modèle simple

Dans cette partie, on va suivre le tutoriel de classification de pytorch (https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).


## Création du Réseau :

Dans le tuto, on utilise la structure suivante :



In [ ]:

class S_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        ## conv1 puis pool :
        x = self.pool(F.relu(self.conv1(x)))
        ## conv2 puis pool :
        x = self.pool(F.relu(self.conv2(x)))
        ## flatten :
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        ## fully connected 1 :
        x = F.relu(self.fc1(x))
        ## fully connected 2 :
        x = F.relu(self.fc2(x))
        ## fully connected 3 :
        x = self.fc3(x)
        return x

    def size_debug(self, x):
        forward_source = inspect.getsource(self.forward)
        forward_lines = forward_source.split('\n')[1:-2]  # Skip def and return
        local_vars = locals()
        print("in :")
        print(x.shape)
        for line in forward_lines:
            if line.strip():
                if line.strip()[0] != '#':
                    exec(line.strip(), globals(), local_vars)
                    x = local_vars['x']
                    print(x.shape)
                elif line.strip()[0:2] == '##':
                    print(line.strip())
        print("out.")


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16


net = S_Net()
net = net.cuda()

net.size_debug(images.cuda())





## Optimization :

Dans le tuto, on utilise comme fonction de permet la Cross Entropy, et comme algorithme d'optimisation, la descente de  gradient stochastique.

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
n_epochs = 2

for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')




In [ ]:
# save the model
PATH = f"./cifar_net_tuto_n_epochs={n_epochs}.pth"
torch.save(net.state_dict(), PATH)

## Evaluation du modèle :

In [ ]:
# tester le modele : (cette case peut être run autant de fois qu'on veut)

dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))


net = Net()
net = net.cuda()
net.load_state_dict(torch.load(PATH))

outputs = net(images.cuda())

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))


In [ ]:
# tester sur tout le test dataset :

net = Net()
net = net.cuda()
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(testset)} test images: {100 * correct // total} %')




In [ ]:
# précision en fonction des classes :

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class

noms_classe = []
precisions_classe = []
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
    noms_classe.append(classname)
    precisions_classe.append(accuracy)
precision_moyenne = np.mean(precisions_classe)


fig, ax = plt.subplots()
ax.bar(noms_classe, precisions_classe)
ax.plot([-0.5,9.5], [precision_moyenne, precision_moyenne], '--r', label='Précision moyenne')
ax.set_xlabel('Classes')
ax.set_ylabel("Précision pour chaque classe")
ax.set_title("Précision en fonction des classes dans l'ensemble de test")
ax.legend()
plt.show()


# Le Modèle PARKHI

In [ ]:
# create the neural network with Pytorch
class P_Net(nn.Module):
  
  def __init__(self, in_channels, out_channels):
    
    super().__init__()

    self.in_channels = in_channels

    S = 32

    self.conv11 = nn.Conv2d(in_channels, S, kernel_size=3, padding=1)
    self.bn11 = nn.BatchNorm2d(S)
    self.conv12 = nn.Conv2d(S, S, kernel_size=3, padding=1)
    self.bn12 = nn.BatchNorm2d(S)

    self.conv21 = nn.Conv2d(S, 2*S, kernel_size=3, padding=1)
    self.bn21 = nn.BatchNorm2d(2*S)
    self.conv22 = nn.Conv2d(2*S, 2*S, kernel_size=3, padding=1)
    self.bn22 = nn.BatchNorm2d(2*S)

    self.conv31 = nn.Conv2d(2*S, 2*2*S, kernel_size=3, padding=1)
    self.bn31 = nn.BatchNorm2d(2*2*S)
    self.conv32 = nn.Conv2d(2*2*S, 2*2*S, kernel_size=3, padding=1)
    self.bn32 = nn.BatchNorm2d(2*2*S)
    self.conv33 = nn.Conv2d(2*2*S, 2*2*S, kernel_size=3, padding=1)
    self.bn33 = nn.BatchNorm2d(2*2*S)

    self.conv41 = nn.Conv2d(2*2*S, 2*2*2*S, kernel_size=3, padding=1)
    self.bn41 = nn.BatchNorm2d(2*2*2*S)
    self.conv42 = nn.Conv2d(2*2*2*S, 2*2*2*S, kernel_size=3, padding=1)
    self.bn42 = nn.BatchNorm2d(2*2*2*S)
    self.conv43 = nn.Conv2d(2*2*2*S, 2*2*2*S, kernel_size=3, padding=1)
    self.bn43 = nn.BatchNorm2d(2*2*2*S)

    self.conv51 = nn.Conv2d(2*2*2*S, 2*2*2*S, kernel_size=3, padding=1)
    self.bn51 = nn.BatchNorm2d(2*2*2*S)
    self.conv52 = nn.Conv2d(2*2*2*S, 2*2*2*S, kernel_size=3, padding=1)
    self.bn52 = nn.BatchNorm2d(2*2*2*S)
    self.conv53 = nn.Conv2d(2*2*2*S, 2*2*2*S, kernel_size=3, padding=1)
    self.bn53 = nn.BatchNorm2d(2*2*2*S)

    self.fc6 = nn.Linear(in_features=2*2*2*S, out_features=4096)
    self.fc7 = nn.Linear(in_features=4096, out_features=4096)  
    self.fc8 = nn.Linear(in_features=4096, out_features=10)


  def forward(self, x):
                  ## Stage 1
    ## conv11 :
    x = F.relu(self.bn11(self.conv11(x)))
    ## conv12 :
    x = F.relu(self.bn12(self.conv12(x)))
    ## maxpool :
    x = F.max_pool2d(x, kernel_size=2, stride=2)
                  ## Stage 2
    ## conv21 :
    x = F.relu(self.bn21(self.conv21(x)))
    ## conv22 :
    x = F.relu(self.bn22(self.conv22(x)))
    ## maxpool :
    x = F.max_pool2d(x, kernel_size=2, stride=2)
                  ## Stage 3
    ## conv31 :
    x = F.relu(self.bn31(self.conv31(x)))
    ## conv32 :
    x = F.relu(self.bn32(self.conv32(x)))
    ## conv33 :
    x = F.relu(self.bn33(self.conv33(x)))
    ## maxpool :
    x = F.max_pool2d(x, kernel_size=2, stride=2)
                  ## Stage 4
    ## conv41 :
    x = F.relu(self.bn41(self.conv41(x)))
    ## conv42 :
    x = F.relu(self.bn42(self.conv42(x)))
    ## conv43 :
    x = F.relu(self.bn43(self.conv43(x)))
    ## maxpool :
    x = F.max_pool2d(x, kernel_size=2, stride=2)
                  ## Stage 5
    ## conv51 :
    x = F.relu(self.bn51(self.conv51(x)))
    ## conv52 :
    x = F.relu(self.bn52(self.conv52(x)))
    ## conv53 :
    x = F.relu(self.bn53(self.conv53(x)))
    ## maxpool :
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    ## flatten :
    x = torch.flatten(x, 1)
                  ## Stage 6
    ## fully connected 6 :
    x = F.relu(self.fc6(x))
                  ## Stage 7
    ## fully connected 7 :
    x = F.relu(self.fc7(x))
                  ## Stage 8
    ## fully connected 8 :
    x = F.relu(self.fc8(x))

    return x
  
  def size_debug(self, x):
        forward_source = inspect.getsource(self.forward)
        forward_lines = forward_source.split('\n')[1:-2]  # Skip def and return
        local_vars = locals()
        print("in :")
        print(x.shape)
        for line in forward_lines:
            if line.strip():
                if line.strip()[0] != '#':
                    exec(line.strip(), globals(), local_vars)
                    x = local_vars['x']
                    print(x.shape)
                elif line.strip()[0:2] == '##':
                    print(line.strip())
        print("out.")

# actual network creation
net = P_Net(3,3)
net.cuda() # go GPU

net.size_debug(images.cuda())

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# # create the optimizer
# optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
# criterion = F.mse_loss # ça marche pas ici parce que il faut criterion(outputs, labels) avec outputs et labels de meme size (pq ??)
# criterion = nn.CrossEntropyLoss()

In [ ]:

n_epochs = 2

for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')



In [ ]:
# save the model
PATH = f"./cifar_net_parkhi_n_epochs={n_epochs}.pth"
torch.save(net.state_dict(), PATH)

In [ ]:
# tester le modele : (cette case peut être run autant de fois qu'on veut)

dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))


net = NetP(3,3)
net.load_state_dict(torch.load(PATH))
#net.cuda()

outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))


In [ ]:
# tester sur tout le test dataset :

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')